## **3. Attivazione e Gradiente MLP**

In [9]:
import math
import random
import torch
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
words = open('./data/names.txt', 'r').read().splitlines()
words[:10]

['emma',
 'olivia',
 'ava',
 'isabella',
 'sophia',
 'charlotte',
 'mia',
 'amelia',
 'harper',
 'evelyn']

In [3]:
# vocabolario di caratteri e mapping a interi
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)

In [6]:
# costruiamo il dataset
def build_dataset(words, n):
    block_size = n
    X, Y = [], []
    for w in words:
        #print(w)
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            #print(''.join(itos[i] for i in context), '-->', itos[ix])
            context = context[1:] + [ix]
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

block_size = 3
X_train, y_train = build_dataset(words[:n1], block_size)
X_valid, y_valid = build_dataset(words[n1:n2], block_size)
X_test, y_test = build_dataset(words[n2:], block_size)

In [7]:
# MLP sistemato:
n_embd = 10
n_hidden = 200

C = torch.randn((vocab_size, n_embd))
W1 = torch.randn((n_embd*block_size, n_hidden))
b1 = torch.randn(n_hidden)
W2 = torch.randn((n_hidden, vocab_size))
b2 = torch.randn((vocab_size))
parameters = [C, W1, b1, W2, b2]
for p in parameters:
    p.requires_grad = True

In [23]:
# Training:
batch_size = 32
for i in range(80000):
    # costruzione minibatch
    ix = torch.randint(0, X_train.shape[0], (batch_size,)) 
    Xb, yb = X_train[ix], y_train[ix]
    # Forward pass:
    emb = C[Xb] # embedding dei caratteri in un vettore
    h = torch.tanh(emb.view(emb.shape[0], n_embd*block_size) @ W1 + b1) # hidden layer
    logits = h @ W2 + b2 # output layer
    loss = F.cross_entropy(logits, yb) # loss function
    # Backward pass:
    for p in parameters:
        p.grad = None
    loss.backward()
    # Update:
    lr = 0.1 if i < 50000 else 0.01
    for p in parameters:
        p.data += -lr * p.grad
print(loss.item())

1.7157104015350342


In [24]:
@torch.no_grad() # disabilita il tracciatore al gradiente
def split_loss(split):
    x,y = {
        'train': (X_train, y_train),
        'val': (X_valid, y_valid),
        'test': (X_test, y_test),
    }[split]
    emb = C[x]
    h = torch.tanh(emb.view(emb.shape[0], n_embd*block_size) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, y)
    print(split, loss.item())
    
split_loss('train')
split_loss('val')
split_loss('test')

train 2.0680699348449707
val 2.115575075149536
test 2.116880178451538


In [25]:
# Genero nomi dal modello:
for i in range(10):
    out = []
    context = [0] * block_size
    while True:
        emb = C[torch.tensor([context])]
        h = torch.tanh(emb.view(emb.shape[0], n_embd*block_size) @ W1 + b1)
        logits = h @ W2 + b2
        probs = F.softmax(logits, dim=1)
        ix = torch.multinomial(probs, num_samples=1, replacement=True).item() # campionamente per ottenere successore
        out.append(ix)
        context = context[1:] + [ix]
        if (ix==0): # se è 0 vuol dire che la parole è terminata
            break

    print(''.join(itos[i] for i in out))

hanie.
shaa.
alevin.
corien.
ryettiah.
jozlyn.
zakayline.
breonter.
tasira.
rosepa.


In [19]:
# La nostra rete non è configurata correttamente nell'inizializzazione:
# all'inizio ci aspetteremo che la rete sia configurata in modo tale che ogni carattere abbia la stessa probabilità di essere il prossimo in quanto non abbiamo dati su cui basarci
1 / 27.0
# se controlliamo la loss function della rete con queste caratteristiche otteniamo 
-torch.tensor(1/27.0).log()
# che è molto minore a quello che abbiamo noi, questo perchè la rete all'inizio da pesi casuali a ogni carattere quindi può capitare che la rete sia erroneamente molto confidente del fatto che un carattere possa essere il prossimo
# vogliamo quindi rendere simili i valori di logits = h @ W2 + b2, possiamo fare ciò diminuendo il valore dei parametri iniziali:
W2 = torch.randn((n_hidden, vocab_size)) * 0.01
b2 = torch.randn((vocab_size)) * 0

In [22]:
# Ora che abbiamo sistemato l'aspetto dei pesi causali iniziali, possiamo notare la presenza di un altro problema:
# usare tanh come funzione di attivazione rende il gradiente da backpropagare non rilevante quando il valore di tanh è vicino ad 1 o -1 ovvero quando siamo vicini alle code
# quindi cambiare gli input in questo caso non impatterà l'output di tanh dato che è in una regione non rilevante
# questo significa che i bias e pesi corrispondenti non impattano la loss
# Nel caso in cui un neurone avesse tutti i valori vicini alle code, esso è un neurone morto e quindi non importa che input prenda, esso non imparerà niente -> NON VOGLIAMO CHE QUESTO ACCADA

# In questo momento la nostra rete ha i neuroni nello strato nascosto con molti valori vicini alle code all'inizializzazione, ma nessuno di essi è morto; quindi non abbiamo grossi problemi
# Nonostante ciò preferiamo diminuire i valori estremi all'inizializzazione in modo tale che la rete neurale sia più reattiva: vogliamo quindi che il valore in input alla funzione di attivazione nello strato nascosto sia più vicino a 0
C = torch.randn((vocab_size, n_embd))
W1 = torch.randn((n_embd*block_size, n_hidden)) * 0.2
b1 = torch.randn(n_hidden) * 0.01
parameters = [C, W1, b1, W2, b2]
for p in parameters:
    p.requires_grad = True